# Fine Tuning

## Introduction

We show how a model can be fine-tuned for a specific task and how the performance can be compared with the pre-trained model.

For this purpose, we want to try to improve the table extraction in the **deep**doctection analyzer as an example. To better understand what we are trying to address, we need to say a little more about processing table extraction.


![title](./pics/dd_table.png)


Table extraction is carried out in different stages:

- Table detection
- Cell detection
- Row and column detection
- Segmentation / cell labeling

Tables, cells and rows / columns are recognized with object detectors (Cascade-RCNN with FPN).
The segmentation is carried out by determining the coverage of cells to rows and columns and is rule-based.

Cell recognition was carried out on the [**PubTabNet**](https://github.com/ibm-aur-nlp/PubTabNet) dataset. PubTabNet contains approx. 500K tables from the field of medical research.

We want to fine-tune cell recognition on a dataset that comes from a completely different domain, namely financial reports. For this we use [**FinTabNet**](https://arxiv.org/pdf/2005.00589.pdf), a data set that contains around 100K tables from that domain.


## Dataset

In order to fine-tune on your own data set, you should create your one dataset instance based on the example of the existing datasets. We use a DatasetRegistry to be able to access the dataset directly. Before we start fine tuning, let's take a look at the dataset. It will show you the advantage of the concept within this framework and how it easily integrates into the training scripts.

In [1]:
import os

from matplotlib import pyplot as plt

from deep_doctection.utils import get_weights_dir_path,get_configs_dir_path
from deep_doctection.datasets import DatasetRegistry
from deep_doctection.mapper import to_page
from deep_doctection.dataflow import MapData
from deep_doctection.train import train_faster_rcnn

In [2]:
DatasetRegistry.print_dataset_names()

['fintabnet', 'funsd', 'testlayout', 'publaynet', 'pubtabnet', 'xfund']


In [3]:
fintabnet = DatasetRegistry.get_dataset("fintabnet")
fintabnet.dataset_info.description

'FinTabNet dataset contains complex tables from the annual reports of S&P 500 companies with detailed table structure annotations to help train and test structure recognition. To generate the cell structure labels, one uses token matching between the PDF and HTML version of each article from public records and filings. Financial tables often have diverse styles when compared to ones in scientific and government documents, with fewer graphical lines and larger gaps within each table and more colour variations. Fintabnet can be used for training cell detection models as well as for semantic table understanding algorithms. For detection it has cell bounding box annotations as well as precisely described table semantics like row - and column numbers and row and col spans. The dataflow builder can also return captions of bounding boxes of rows and columns. Moreover, various filter conditions on the table structure are available: maximum cell numbers, maximal row and column numbers and their

We refer to the in depths tutorial for more details about the construction of datasets and the architecture of **deep**doctection. Nevertheless, we will briefly go into the individual steps to display a sample from Fintabnet. The dataset has a method dataflow.build that returns a generator where samples can be streamed from. 

In [4]:
df = fintabnet.dataflow.build(split="train",load_image=True,use_multi_proc=False)
df = MapData(df,to_page)
df.reset_state()
df_iter = iter(df)

[1216 18:55.28 @fintabnet.py:160] INF Logic will currently display only ONE table per page, even if there are more !!


1.) Datasets have dataflow components. Dataflows allow efficient loading and mapping of data and thus represent the bloodstream of deep doctection. The build method creates a dataflow of the dataset. By selecting certain parameters, for example, a split can be selected or it can be determined whether the underlying image should be loaded.

2.) In the second line, the core data model is mapped to an easily consumable page object. Parsed results can be queried and visualized in the page object.

3.) The reset_state () method of the dataflow must be called before iterating the dataflow and belongs to the Dataflow API.

4.) We want to use the next method to look at samples, so we create an iterator.

After we have created a page object, we enter the annotations in the image with viz () and visualize them.

In [ ]:
table=next(df_iter)
image = table.viz()
plt.figure(figsize = (20,10))
plt.axis('off')
plt.imshow(image)

![title](https://github.com/deepdoctection/deepdoctection/raw/master/docs/tutorials/pics/output_7_1.png)

The sample shows that tables are not cropped from the image. Our method however requires that the cell predictor requires tables only without surronding. The build method for this particular dataset, however has a optional parameter `build_mode` such that the dataflow returns tables only, once you pass the value 'table'. The coordinates of the cells are also converted to the coordinate system of the cropped image.


## Training Tensorpack Predictor

For the training, we use a training script which stems from the training of the Faster-RCNN model from Tensorpack. Let's collect all necessary inputs:

- We take the model config for the cell detector. It is important to note that the hyperparameter for this detector differs slightly from the standard Faster-RCNN config for taking into account that cells are generally     smaller and have in general a length/height ratio >=1.
- We take the pre-trained cell weights.
- We choose fintabnet for training and evaluation for which the datasets is pre-splitted.
- Config overwrites specifies the train setting. We define 1 epoch to have 500 pass throughs. (Note, that the term of one epoch is different than in most of the literature.) We set the learning rate to be .001 which is quite common for fine tuning this model. We set the schedule to have 50K iterations and set a random resizing of the train dataset between 600 and 1.2K pixels.
- The other settings affect the dataset dataflow build config. As already mentioned we need to crop tables from the images. As we crop them at the beginning, we have to store these images in memory. This is not ideal but peculiar for this dataset and requires either a large memory or a small training dataset. We choose 15k samples. As we only use parts of the images this might ok, but you can set this setting lower if you RAM is not that large.  

In [6]:
path_config_yaml=os.path.join(get_configs_dir_path(),"tp/cell/conf_frcnn_cell.yaml")
path_weights = os.path.join(get_weights_dir_path(),"cell/model-2840000.data-00000-of-00001")

dataset_train = fintabnet

config_overwrite=["TRAIN.STEPS_PER_EPOCH=500","TRAIN.EVAL_PERIOD=20","TRAIN.BASE_LR=1e-3","TRAIN.LR_SCHEDULE=[50000]","PREPROC.TRAIN_SHORT_EDGE_SIZE=[600,1200]"]
build_train_config=["build_mode=table","load_image=True","max_datapoints=10000","use_multi_proc=False"]
dataset_val = fintabnet
build_val_config = ["build_mode=table","load_image=True","max_datapoints=1000","use_multi_proc=False"]

Finally, we set a directory to save the checkpoints and log the training progress. For evaluation we use the traditional coco metric which returns mean average precision and mean average recall.

In [ ]:
train_faster_rcnn(path_config_yaml=path_config_yaml,
                  dataset_train=fintabnet,
                  path_weights=path_weights,
                  config_overwrite=config_overwrite,
                  log_dir="/path/to/dir/test_train",
                  build_train_config=build_train_config,
                  dataset_val=dataset_val,
                  build_val_config=build_val_config,
                  metric_name="coco",
                  pipeline_component_name="ImageLayoutService"
                  )

[1216 18:55.36 @logger.py:93] INF Argv: ['/usr/lib/python3/dist-packages/ipykernel_launcher.py', '-f', '/home/janis/.local/share/jupyter/runtime/kernel-60921085-da62-4658-8f59-98ccb0e43f95.json'] 
[1216 18:55.37 @config.py:320] INF Environment Information:
 -----------------------  ------------------------------------------------------------------------------------------------
sys.platform             linux
Python                   3.8.10 (default, Sep 28 2021, 16:10:42) [GCC 9.3.0]
Tensorpack               v0.11-0-gdb541e8e @/home/janis/Public/deepdoctection/venv/lib/python3.8/site-packages/tensorpack
Numpy                    1.21.4
TensorFlow               2.4.1/unknown @/usr/lib/python3/dist-packages/tensorflow
TF Compiler Version      9.3.0
TF CUDA support          True
TF MKL support           False
TF XLA support           False
Nvidia Driver            /usr/lib/x86_64-linux-gnu/libnvidia-ml.so.460.73.01
CUDA libs                /usr/lib/x86_64-linux-gnu/libcudart.so.11.1.74
CUDN

  0%|          |0/10000[00:00<?,?it/s]

[1216 18:55.37 @logger.py:193] WRN Datapoint have images as np arrays stored and they will be loaded into memory. To avoid OOM set 'load_image'=False in dataflow build config. This will load images when needed and reduce memory costs!!!


 52%|#####2    |5212/10000[25:33<34:15, 2.33it/s]

[1216 19:21.12 @logger.py:193] INF dp: 13058, err: AssertionError,
            msg: bounding box must have height and width >0. Check coords ulx: 526.05, uly: 517.95, lrx: 22.71, lry: 419.73. in: <frame at 0x7280810, file '/home/janis/Public/deepdoctection/deep_doctection/mapper/pubstruct.py', line 355, code pub_to_image_uncur> will be filtered


...

100%|##########|500/500[01:22<00:00, 6.08it/s]

[1216 20:38:47 @base.py:282] Epoch 25 (global_step 2852500) finished, time:1 minute 22 seconds.
[1216 20:38:47 @monitor.py:476] QueueInput/queue_size: 50
[1216 20:38:47 @monitor.py:476] cascade_loss_stage1/fastrcnn_losses/box_loss: 0.05577
[1216 20:38:47 @monitor.py:476] cascade_loss_stage1/fastrcnn_losses/label_loss: 0.26561
[1216 20:38:47 @monitor.py:476] cascade_loss_stage1/fastrcnn_losses/label_metrics/accuracy: 0.92726
[1216 20:38:47 @monitor.py:476] cascade_loss_stage1/fastrcnn_losses/label_metrics/false_negative: 0.29591
[1216 20:38:47 @monitor.py:476] cascade_loss_stage1/fastrcnn_losses/label_metrics/fg_accuracy: 0.4985
[1216 20:38:47 @monitor.py:476] cascade_loss_stage1/fastrcnn_losses/num_fg_label: 67.341
[1216 20:38:47 @monitor.py:476] cascade_loss_stage2/fastrcnn_losses/box_loss: 0.084868
[1216 20:38:47 @monitor.py:476] cascade_loss_stage2/fastrcnn_losses/label_loss: 0.163
[1216 20:38:47 @monitor.py:476] cascade_loss_stage2/fastrcnn_losses/label_metrics/accuracy: 0.95702
[1


100%|##########|500/500[01:24<00:00, 5.95it/s]

[1216 20:40:11 @base.py:282] Epoch 26 (global_step 2853000) finished, time:1 minute 24 seconds.
[1216 20:40:11 @monitor.py:476] QueueInput/queue_size: 50
[1216 20:40:11 @monitor.py:476] cascade_loss_stage1/fastrcnn_losses/box_loss: 0.035898
[1216 20:40:11 @monitor.py:476] cascade_loss_stage1/fastrcnn_losses/label_loss: 0.19599
[1216 20:40:11 @monitor.py:476] cascade_loss_stage1/fastrcnn_losses/label_metrics/accuracy: 0.94674
[1216 20:40:11 @monitor.py:476] cascade_loss_stage1/fastrcnn_losses/label_metrics/false_negative: 0.15908
[1216 20:40:11 @monitor.py:476] cascade_loss_stage1/fastrcnn_losses/label_metrics/fg_accuracy: 0.57348
[1216 20:40:11 @monitor.py:476] cascade_loss_stage1/fastrcnn_losses/num_fg_label: 55.653
[1216 20:40:11 @monitor.py:476] cascade_loss_stage2/fastrcnn_losses/box_loss: 0.05223
[1216 20:40:11 @monitor.py:476] cascade_loss_stage2/fastrcnn_losses/label_loss: 0.10567
[1216 20:40:11 @monitor.py:476] cascade_loss_stage2/fastrcnn_losses/label_metrics/accuracy: 0.97314


 43%|####3     |215/500[00:35<00:45, 6.23it/s]